In [9]:
import io
import numpy as np
import PIL
from PIL import Image
import requests
import torch
import openpifpaf
from os import listdir
from os.path import isfile, join
from tensorflow import keras
import os

In [22]:
fpath = "internet_pics/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "pictures found.")

8059 pictures found.


In [23]:
i = 0
os.makedirs("processed/", exist_ok=True)
for f in allfiles:
    pil_im = Image.open(fpath + f).convert("RGB")
    im = np.asarray(pil_im)
    predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')
    predictions, gt_anns, image_meta = predictor.pil_image(pil_im)
    if len(predictions) == 0: continue
    
    for prediction in predictions:
    
        if prediction.score < 0.65: continue
        i += 1
        bbox = prediction.bbox()

        padding = 0.1
        l_x = bbox[0] - padding * bbox[2]
        l_y = bbox[1] - padding * bbox[3]
        r_x = l_x + (1 + 2 * padding) * bbox[2]
        r_y = l_y + (1 + 2 * padding) * bbox[3]

        if l_x < 0: l_x = 0
        if l_y < 0: l_y = 0
        if r_x > pil_im.width: r_x = pil_im.width
        if r_y > pil_im.height: r_y = pil_im.height

        cropped = pil_im.crop((l_x, l_y, r_x, r_y))
        cropped = cropped.resize((100, 200))

        cropped.save(f"processed/procd_{str(i)}.png")
        print("Image with {} confidence saved".format(prediction.score))
        i += 1

Image with 0.7439591780952785 confidence saved
Image with 0.7113143931264464 confidence saved
Image with 0.7172190417414126 confidence saved
Image with 0.8425769494927446 confidence saved
Image with 0.7434096180874368 confidence saved
Image with 0.769831315330837 confidence saved
Image with 0.9022997876872186 confidence saved
Image with 0.8223948141802913 confidence saved
Image with 0.8732448038847549 confidence saved
Image with 0.9227917738582777 confidence saved
Image with 0.8722849410513173 confidence saved
Image with 0.7875062745550404 confidence saved
Image with 0.6683804677880328 confidence saved
Image with 0.7763626653215159 confidence saved
Image with 0.8460677071757938 confidence saved
Image with 0.9086783113686935 confidence saved
Image with 0.6547421683435855 confidence saved
Image with 0.9088936525842417 confidence saved
Image with 0.6674998739491338 confidence saved
Image with 0.780866845794346 confidence saved
Image with 0.8487366800722869 confidence saved
Image with 0.73

Image with 0.9037274277728536 confidence saved
Image with 0.7957116080367047 confidence saved
Image with 0.8032015665717747 confidence saved
Image with 0.6957514156465945 confidence saved
Image with 0.7765201226524685 confidence saved
Image with 0.7281586450079213 confidence saved
Image with 0.822660339915234 confidence saved
Image with 0.8622699224430582 confidence saved
Image with 0.893015884834787 confidence saved
Image with 0.8782908657322759 confidence saved
Image with 0.8699244986409727 confidence saved
Image with 0.7553813276083572 confidence saved
Image with 0.7553813276083572 confidence saved
Image with 0.7610803764799365 confidence saved
Image with 0.7529625555743341 confidence saved
Image with 0.7415053481641023 confidence saved
Image with 0.6930526183999103 confidence saved
Image with 0.8721272997234178 confidence saved
Image with 0.873517746510713 confidence saved
Image with 0.8107332872307819 confidence saved
Image with 0.8467121357503145 confidence saved
Image with 0.807

Image with 0.7848234461701434 confidence saved
Image with 0.7510031539460887 confidence saved
Image with 0.6772935157236846 confidence saved
Image with 0.6942888731541841 confidence saved
Image with 0.6560422233913255 confidence saved
Image with 0.8400769492854243 confidence saved
Image with 0.8078226628510848 confidence saved
Image with 0.6609474394632423 confidence saved
Image with 0.7356022259463434 confidence saved
Image with 0.9109337718590447 confidence saved
Image with 0.8548625578051028 confidence saved
Image with 0.8570324985877326 confidence saved
Image with 0.8589841282886006 confidence saved
Image with 0.8210158762724503 confidence saved
Image with 0.8649255866589753 confidence saved
Image with 0.9418661672136057 confidence saved
Image with 0.9129867087239805 confidence saved
Image with 0.8559323173502217 confidence saved
Image with 0.7553028697552888 confidence saved
Image with 0.9002549544624661 confidence saved
Image with 0.8728166455807893 confidence saved
Image with 0.

Image with 0.9407926642376443 confidence saved
Image with 0.9099616615668586 confidence saved
Image with 0.8830759188403253 confidence saved
Image with 0.7651640824649644 confidence saved
Image with 0.6908703731453938 confidence saved
Image with 0.868909602579863 confidence saved
Image with 0.7789538502693176 confidence saved
Image with 0.8828417192334714 confidence saved
Image with 0.6733661034832831 confidence saved
Image with 0.7076117629590242 confidence saved
Image with 0.7046095573383828 confidence saved
Image with 0.7180802562962407 confidence saved
Image with 0.9165399022724317 confidence saved
Image with 0.8389016338016676 confidence saved
Image with 0.7079430585322173 confidence saved
Image with 0.7501117820325105 confidence saved
Image with 0.693364477675894 confidence saved
Image with 0.6737806771112524 confidence saved
Image with 0.6753031585527502 confidence saved
Image with 0.6812583182169043 confidence saved
Image with 0.8969012602515842 confidence saved
Image with 0.89

Image with 0.7010534431623376 confidence saved
Image with 0.8978144526481628 confidence saved
Image with 0.6903911310693491 confidence saved
Image with 0.9194762032964955 confidence saved
Image with 0.8480906279190727 confidence saved
Image with 0.8677677745404451 confidence saved
Image with 0.8572301009426946 confidence saved
Image with 0.6523333476937335 confidence saved
Image with 0.8850753488747969 confidence saved
Image with 0.7737168358719866 confidence saved
Image with 0.7990564172682555 confidence saved
Image with 0.7750658081925434 confidence saved
Image with 0.7676173707713252 confidence saved
Image with 0.8107697808224221 confidence saved
Image with 0.6846022035764612 confidence saved
Image with 0.6588213184605474 confidence saved
Image with 0.8057387108388153 confidence saved
Image with 0.783678438352502 confidence saved
Image with 0.8195364345674929 confidence saved
Image with 0.9178205573040507 confidence saved
Image with 0.6755017741866733 confidence saved
Image with 0.7

...

Image with 0.7547268193701039 confidence saved
Image with 0.7279806111169898 confidence saved
Image with 0.6795762492262798 confidence saved
Image with 0.893009048441182 confidence saved
Image with 0.9411534537439761 confidence saved
Image with 0.712786184704822 confidence saved
Image with 0.6630310649457184 confidence saved
Image with 0.9376727575841157 confidence saved
Image with 0.7210547483485678 confidence saved
Image with 0.826633790264959 confidence saved
Image with 0.940316062906514 confidence saved
Image with 0.6941573438437088 confidence saved
Image with 0.7468173970346865 confidence saved
Image with 0.8510865076728489 confidence saved
Image with 0.6606618554695792 confidence saved
Image with 0.660768184972846 confidence saved
Image with 0.8318578497223232 confidence saved
Image with 0.9025753166364586 confidence saved
Image with 0.9238854050636292 confidence saved
Image with 0.8615524535593779 confidence saved
Image with 0.7227677314177803 confidence saved
Image with 0.72696

Image with 0.8454201221466063 confidence saved
Image with 0.8686469482338947 confidence saved
Image with 0.7724804930064989 confidence saved
Image with 0.7132444874100063 confidence saved
Image with 0.690815868585006 confidence saved
Image with 0.669986455336861 confidence saved
Image with 0.6888477569041044 confidence saved
Image with 0.8840536293776139 confidence saved
Image with 0.8804962453634841 confidence saved
Image with 0.9526247744974884 confidence saved
Image with 0.7020096403101216 confidence saved
Image with 0.9271129939867102 confidence saved
Image with 0.6669022207674773 confidence saved
Image with 0.6786513846853505 confidence saved
Image with 0.7740122235339622 confidence saved
Image with 0.7718002407447151 confidence saved
Image with 0.7734194993972778 confidence saved
Image with 0.8233554674231489 confidence saved
Image with 0.7769091077472852 confidence saved
Image with 0.8964628810467926 confidence saved
Image with 0.8226405485816622 confidence saved
Image with 0.85

Image with 0.8170283540435459 confidence saved
Image with 0.8044513308483622 confidence saved
Image with 0.8083415575649427 confidence saved
Image with 0.6676218742909638 confidence saved
Image with 0.7720255009506058 confidence saved
Image with 0.7298078303751737 confidence saved
Image with 0.743987923083098 confidence saved
Image with 0.915037453174591 confidence saved
Image with 0.8645672746326611 confidence saved
Image with 0.7130749549554742 confidence saved
Image with 0.7619276189285775 confidence saved
Image with 0.8751634126124174 confidence saved
Image with 0.8682134462439495 confidence saved
Image with 0.7251575330029363 confidence saved
Image with 0.7752563098202581 confidence saved
Image with 0.7874825259913569 confidence saved
Image with 0.7043405641680178 confidence saved
Image with 0.8036872433579486 confidence saved
Image with 0.6803196668624878 confidence saved
Image with 0.6796424751696379 confidence saved
Image with 0.8529006501902704 confidence saved
Image with 0.89

Image with 0.7713829797247183 confidence saved
Image with 0.7382353051849033 confidence saved
Image with 0.8807620820791825 confidence saved
Image with 0.8872041818888291 confidence saved
Image with 0.8844215118366737 confidence saved
Image with 0.7654498841451561 confidence saved
Image with 0.6567564503006312 confidence saved
Image with 0.7287440896034241 confidence saved
Image with 0.6652627639148546 confidence saved
Image with 0.8285700393759685 confidence saved
Image with 0.9048659516417463 confidence saved
Image with 0.8955315429231395 confidence saved
Image with 0.7768689808638198 confidence saved
Image with 0.7690469259801118 confidence saved
Image with 0.9100717928098596 confidence saved
Image with 0.8747772989065752 confidence saved
Image with 0.6779433061247286 confidence saved
Image with 0.8005585191042526 confidence saved
Image with 0.7461921183959298 confidence saved
Image with 0.7411392175632974 confidence saved
Image with 0.7182629406452179 confidence saved
Image with 0.

Image with 0.7744419807973115 confidence saved
Image with 0.8537029053853905 confidence saved
Image with 0.7451996855113818 confidence saved
Image with 0.8818013253419295 confidence saved
Image with 0.6956158254457556 confidence saved
Image with 0.7933806427147078 confidence saved
Image with 0.8319102111070054 confidence saved
Image with 0.7039156607959581 confidence saved
Image with 0.7083187362422113 confidence saved
Image with 0.8082018131795137 confidence saved
Image with 0.7084412730258444 confidence saved
Image with 0.8099326478398364 confidence saved
Image with 0.7998898443968399 confidence saved
Image with 0.6822980357253032 confidence saved
Image with 0.8395365943079409 confidence saved
Image with 0.8402644214422805 confidence saved
Image with 0.7219141332999519 confidence saved
Image with 0.8364079698272373 confidence saved
Image with 0.7258259705875231 confidence saved
Image with 0.8206918291423632 confidence saved
Image with 0.6887805124987727 confidence saved
Image with 0.

In [10]:
def predict_smato(cropped_image, model):
    '''Returns True (has smartphone) or False based on the model
    '''
    im = np.asarray(cropped_image) * 1.0/255.0
    im = im.reshape((1, 224, 224, 3))
    prediction = model.predict(im)[0][0]
    print("Raw prediction results: ", prediction)
    return prediction >= 0.5

In [11]:
# load model
model_path = "saved_models/smato_efficientnet_m2/"
model = keras.models.load_model(model_path)

In [26]:
fpath = "processed/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "processed pictures found.")

5405 processed pictures found.


In [27]:
# sort into two different folders basde on trained model
# this reduces time to manually sort them
# after this, manually fix the classes
i = 0
for f in allfiles:
    cropped_image = Image.open(fpath + f).convert("RGB").resize((224, 224))
    result = predict_smato(cropped_image, model)
    print("Holding smartphone: ", result)
    cropped_image = cropped_image.resize((100, 200))
    if result:
        cropped_image.save(f"with/true_{i}.png")
    else:
        cropped_image.save(f"without/false_{i}.png")
    i += 1

Raw prediction results:  0.28307563
Holding smartphone:  False
Raw prediction results:  0.011569644
Holding smartphone:  False
Raw prediction results:  0.8605806
Holding smartphone:  True
Raw prediction results:  0.8092411
Holding smartphone:  True
Raw prediction results:  0.053518873
Holding smartphone:  False
Raw prediction results:  0.962458
Holding smartphone:  True
Raw prediction results:  0.99880934
Holding smartphone:  True
Raw prediction results:  0.7428251
Holding smartphone:  True
Raw prediction results:  0.052002504
Holding smartphone:  False
Raw prediction results:  0.9153732
Holding smartphone:  True
Raw prediction results:  0.9576043
Holding smartphone:  True
Raw prediction results:  0.004070876
Holding smartphone:  False
Raw prediction results:  8.305514e-06
Holding smartphone:  False
Raw prediction results:  0.79292107
Holding smartphone:  True
Raw prediction results:  0.04540568
Holding smartphone:  False
Raw prediction results:  0.58241564
Holding smartphone:  True
Ra

Raw prediction results:  0.03159905
Holding smartphone:  False
Raw prediction results:  0.24595551
Holding smartphone:  False
Raw prediction results:  0.0030387235
Holding smartphone:  False
Raw prediction results:  3.3331056e-05
Holding smartphone:  False
Raw prediction results:  0.0072028013
Holding smartphone:  False
Raw prediction results:  0.9991948
Holding smartphone:  True
Raw prediction results:  0.6309581
Holding smartphone:  True
Raw prediction results:  0.9926306
Holding smartphone:  True
Raw prediction results:  0.97051424
Holding smartphone:  True
Raw prediction results:  0.39193296
Holding smartphone:  False
Raw prediction results:  0.08189487
Holding smartphone:  False
Raw prediction results:  0.99878126
Holding smartphone:  True
Raw prediction results:  0.01642424
Holding smartphone:  False
Raw prediction results:  0.9340584
Holding smartphone:  True
Raw prediction results:  2.3772953e-05
Holding smartphone:  False
Raw prediction results:  0.986904
Holding smartphone:  

Raw prediction results:  0.9957313
Holding smartphone:  True
Raw prediction results:  0.02930136
Holding smartphone:  False
Raw prediction results:  0.8927665
Holding smartphone:  True
Raw prediction results:  0.0023272
Holding smartphone:  False
Raw prediction results:  0.98528904
Holding smartphone:  True
Raw prediction results:  0.010720168
Holding smartphone:  False
Raw prediction results:  0.99914074
Holding smartphone:  True
Raw prediction results:  0.94424355
Holding smartphone:  True
Raw prediction results:  0.02164651
Holding smartphone:  False
Raw prediction results:  0.02552377
Holding smartphone:  False
Raw prediction results:  0.9990476
Holding smartphone:  True
Raw prediction results:  0.00014040954
Holding smartphone:  False
Raw prediction results:  0.9987363
Holding smartphone:  True
Raw prediction results:  7.3775022e-06
Holding smartphone:  False
Raw prediction results:  0.013525345
Holding smartphone:  False
Raw prediction results:  6.772337e-06
Holding smartphone:  

Raw prediction results:  0.9917503
Holding smartphone:  True
Raw prediction results:  0.999064
Holding smartphone:  True
Raw prediction results:  6.557523e-05
Holding smartphone:  False
Raw prediction results:  4.111278e-06
Holding smartphone:  False
Raw prediction results:  0.118585914
Holding smartphone:  False
Raw prediction results:  0.99876803
Holding smartphone:  True
Raw prediction results:  0.9608576
Holding smartphone:  True
Raw prediction results:  0.98766685
Holding smartphone:  True
Raw prediction results:  0.99720204
Holding smartphone:  True
Raw prediction results:  0.17028661
Holding smartphone:  False
Raw prediction results:  0.020032402
Holding smartphone:  False
Raw prediction results:  0.0058965445
Holding smartphone:  False
Raw prediction results:  0.010629479
Holding smartphone:  False
Raw prediction results:  0.0041678213
Holding smartphone:  False
Raw prediction results:  0.19996384
Holding smartphone:  False
Raw prediction results:  0.08399841
Holding smartphone

Raw prediction results:  0.4931897
Holding smartphone:  False
Raw prediction results:  0.00087924383
Holding smartphone:  False
Raw prediction results:  0.9930636
Holding smartphone:  True
Raw prediction results:  0.9999591
Holding smartphone:  True
Raw prediction results:  0.01862843
Holding smartphone:  False
Raw prediction results:  0.9947837
Holding smartphone:  True
Raw prediction results:  0.9675619
Holding smartphone:  True
Raw prediction results:  0.93823373
Holding smartphone:  True
Raw prediction results:  0.34449643
Holding smartphone:  False
Raw prediction results:  0.008840771
Holding smartphone:  False
Raw prediction results:  0.00823632
Holding smartphone:  False
Raw prediction results:  0.99760383
Holding smartphone:  True
Raw prediction results:  0.99631786
Holding smartphone:  True
Raw prediction results:  0.9987514
Holding smartphone:  True
Raw prediction results:  0.9997286
Holding smartphone:  True
Raw prediction results:  0.12819366
Holding smartphone:  False
Raw 

...

Raw prediction results:  0.9706752
Holding smartphone:  True
Raw prediction results:  0.9996767
Holding smartphone:  True
Raw prediction results:  0.62866324
Holding smartphone:  True
Raw prediction results:  0.9643954
Holding smartphone:  True
Raw prediction results:  0.99723274
Holding smartphone:  True
Raw prediction results:  0.97936136
Holding smartphone:  True
Raw prediction results:  0.9970379
Holding smartphone:  True
Raw prediction results:  0.08829173
Holding smartphone:  False
Raw prediction results:  0.75124323
Holding smartphone:  True
Raw prediction results:  0.97685224
Holding smartphone:  True
Raw prediction results:  0.019118568
Holding smartphone:  False
Raw prediction results:  0.98843944
Holding smartphone:  True
Raw prediction results:  0.99293983
Holding smartphone:  True
Raw prediction results:  0.9757966
Holding smartphone:  True
Raw prediction results:  0.112803705
Holding smartphone:  False
Raw prediction results:  0.01026883
Holding smartphone:  False
Raw pre

Raw prediction results:  0.9861616
Holding smartphone:  True
Raw prediction results:  0.09183603
Holding smartphone:  False
Raw prediction results:  0.0009205482
Holding smartphone:  False
Raw prediction results:  0.999954
Holding smartphone:  True
Raw prediction results:  2.0133511e-07
Holding smartphone:  False
Raw prediction results:  0.99888533
Holding smartphone:  True
Raw prediction results:  0.99846965
Holding smartphone:  True
Raw prediction results:  0.99998796
Holding smartphone:  True
Raw prediction results:  0.9923523
Holding smartphone:  True
Raw prediction results:  7.1133065e-05
Holding smartphone:  False
Raw prediction results:  0.00014400162
Holding smartphone:  False
Raw prediction results:  0.002652609
Holding smartphone:  False
Raw prediction results:  0.99187225
Holding smartphone:  True
Raw prediction results:  0.025273224
Holding smartphone:  False
Raw prediction results:  0.7042972
Holding smartphone:  True
Raw prediction results:  0.98639166
Holding smartphone:

Raw prediction results:  0.9821176
Holding smartphone:  True
Raw prediction results:  0.99345946
Holding smartphone:  True
Raw prediction results:  0.023676522
Holding smartphone:  False
Raw prediction results:  0.15482044
Holding smartphone:  False
Raw prediction results:  0.9958281
Holding smartphone:  True
Raw prediction results:  0.7885387
Holding smartphone:  True
Raw prediction results:  0.0009812654
Holding smartphone:  False
Raw prediction results:  1.6889805e-06
Holding smartphone:  False
Raw prediction results:  0.001287904
Holding smartphone:  False
Raw prediction results:  0.947249
Holding smartphone:  True
Raw prediction results:  0.0026433794
Holding smartphone:  False
Raw prediction results:  0.99999976
Holding smartphone:  True
Raw prediction results:  0.507794
Holding smartphone:  True
Raw prediction results:  0.98400867
Holding smartphone:  True
Raw prediction results:  0.96291417
Holding smartphone:  True
Raw prediction results:  0.97171754
Holding smartphone:  True


Raw prediction results:  0.0006786065
Holding smartphone:  False
Raw prediction results:  0.9991462
Holding smartphone:  True
Raw prediction results:  0.99356383
Holding smartphone:  True
Raw prediction results:  0.010363006
Holding smartphone:  False
Raw prediction results:  0.97179574
Holding smartphone:  True
Raw prediction results:  0.9998615
Holding smartphone:  True
Raw prediction results:  0.9960472
Holding smartphone:  True
Raw prediction results:  0.9968887
Holding smartphone:  True
Raw prediction results:  0.9407812
Holding smartphone:  True
Raw prediction results:  0.050698746
Holding smartphone:  False
Raw prediction results:  0.042795632
Holding smartphone:  False
Raw prediction results:  0.8971141
Holding smartphone:  True
Raw prediction results:  0.015619254
Holding smartphone:  False
Raw prediction results:  0.9999007
Holding smartphone:  True
Raw prediction results:  0.003440487
Holding smartphone:  False
Raw prediction results:  0.99920964
Holding smartphone:  True
Ra

Raw prediction results:  0.034099467
Holding smartphone:  False
Raw prediction results:  0.9617906
Holding smartphone:  True
Raw prediction results:  0.1666282
Holding smartphone:  False
Raw prediction results:  0.46714237
Holding smartphone:  False
Raw prediction results:  0.005275005
Holding smartphone:  False
Raw prediction results:  0.039790656
Holding smartphone:  False
Raw prediction results:  0.006104505
Holding smartphone:  False
Raw prediction results:  0.7071209
Holding smartphone:  True
Raw prediction results:  0.0040845573
Holding smartphone:  False
Raw prediction results:  0.004636049
Holding smartphone:  False
Raw prediction results:  0.004470879
Holding smartphone:  False
Raw prediction results:  0.99990606
Holding smartphone:  True
Raw prediction results:  0.0006995361
Holding smartphone:  False
Raw prediction results:  0.98768556
Holding smartphone:  True
Raw prediction results:  0.9991934
Holding smartphone:  True
Raw prediction results:  0.9967449
Holding smartphone:

In [28]:
# for each of the classes, add flipped images to augment data
fpath = "with/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "with smartphone pictures found.")

for f in allfiles:
    original_img = Image.open(fpath + f)
    horz_img = original_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    horz_img.save(fpath + "flipped_" + f)

301 with smartphone pictures found.


In [21]:
# for each of the classes, add flipped images to augment data
fpath = "without/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "without smartphone pictures found.")

for f in allfiles:
    original_img = Image.open(fpath + f)
    horz_img = original_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    horz_img.save(fpath + "flipped_" + f)

125 without smartphone pictures found.
